Дипломная работа на курсе МГТУ им. Н. Баумана

Проект по разработке нейронной сети по определению характеристик компазиционных материалов
Версия 1.
Начало.

Подготовка среды для проекта ENV
При начале работы над проектом необходимо было установить несколько пакетов библиотек Python таких как tensorflow, seaborn и др.
Выяснилось, что невозможно установить пакеты через PIP install, так как win 10 не воспринимат путь к файлу длиннее 260 симвлов. Пришлось написть программу по изменению Политики доступа для win 10 с расширением .bat. Однако это тоже не дало результатов. Пришлось с помощью изменения regedit вносить изменения в ключь Computer\HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem, что привело к возможности записывать блинные пути к файлу.

Но! Установка tensorflow прошла только для моего окружения  MeEnvJupiterLab!!!!!  
  WARNING: The scripts estimator_ckpt_converter.exe, import_pb_to_tensorboard.exe, saved_model_cli.exe, tensorboard.exe, tf_upgrade_v2.exe, tflite_convert.exe, toco.exe and toco_from_protos.exe are installed in 'C:\Users\grain\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\Scripts' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Successfully installed tensorflow-2.11.0 tensorflow-intel-2.11.0

In [1]:
# Импортируем в проект требуемые библиотеки

import pandas as pd # библиотека для обработки и анализа данных
import numpy as np
import seaborn as sns # библиотека для создания статистических графиков
import random #  генераторатор случайных чисел и данных

import googletrans
from googletrans import Translator

CRISP-DM – Cross-Industry Standard Process for Data Mining
CRISP-DM описывает жизненный цикл исследования данных, состоящий из 6 фаз/этапов.



# Этап 1 Business Understanding / Бизнес анализ

Композиционные материалы - это искусственно созданные материалы, состоящие из нескольких других с четкой границей между ними. Композиты обладают теми свойствами, которые не наблюдаются у компонентов по отдельности. При этом композиты являются монолитным материалом, т.е. компоненты материала неотделимы друг от друга без разрушения конструкции в целом. Яркий пример композита - железобетон. Бетон прекрасно сопротивляется сжатию, но плохо растяжению. Стальная арматура внутри бетона компенсирует его неспособность сопротивляться сжатию, формируя тем самым новые, уникальные свойства. Современные композиты изготавливаются из других материалов: полимеры, керамика, стеклянные и углеродные волокна, но данный принцип сохраняется. У такого подхода есть и недостаток: даже если мы знаем характеристики исходных компонентов, определить характеристики композита, состоящего из этих компонентов, достаточно проблематично. Для решения этой проблемы есть два пути: физические испытания образцов материалов, или прогнозирование характеристик. Суть прогнозирования заключается в симуляции представительного элемента объема композита, на основе данных о характеристиках входящих компонентов (связующего и армирующего компонента).
На входе имеются данные о начальных свойствах компонентов композиционных материалов (количество связующего, наполнителя, температурный режим отверждения и т.д.). На выходе необходимо спрогнозировать ряд конечных свойств получаемых композиционных материалов. Кейс основан на реальных производственных задачах Центра НТИ «Цифровое материаловедение: новые материалы и вещества» (структурное подразделение МГТУ им. Н.Э. Баумана).


# Этап 2 Data Understanding / Анализ данных

# Этап 2.1. Collect Initial Data/ Сбор данных

Загружаем DataSet предложеный  на курсе (он состоит из 2-х файлов) по ссылке:
https://drive.google.com/file/d/1B1s5gBlvgU81H9GGolLQVw_SOi-vyNf2/view?usp=sharing
При распковки DataSet получались 2 файла формата excel. При открытии этих файлов в JupiterLab выдается ошибка:
File Load Error for X_nup.xlsx
C:\Users\grain\Work_folder\Diplom_MGTU\data_diplom\X_nup.xlsx is not UTF-8 encoded
Открыв файлы в программе Excel попытался записать исходные файлы с расширением .csv в кодировке "UTF-8, разделитель запятая".
При попытке прочтения этих файлов с расширением .csv в коде в Jupiter df_bp = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\x_bp.csv') все равно получается ошибка:
ParserError: Error tokenizing data. C error: Expected 10 fields in line 25, saw 11
Тогда я прочитал DataSet в Jupiternotebook как файл Excel: df_bp = pd.read_excel()


# Этап 2.2. Discribe Data / Описание данных и Verify Data Quality / Проверка качества данных

In [2]:
# считываем 1-й файл X_bp.xlsx
df_bp = pd.read_excel(r'C:\Users\grain\Work_folder\Diplom_MGTU\data_diplom\X_bp.xlsx') 

In [3]:
df_bp.head() # выводим df_bp

,Unnamed: 0,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2"
0,0,1.857143,2030.0,738.736842,30.00,22.267857,100.000000,210.0,70.0,3000.0,220.0
1,1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0
2,2,1.857143,2030.0,738.736842,49.90,33.000000,284.615385,210.0,70.0,3000.0,220.0
3,3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0
4,4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0


Файл Excel прочитался, во всех колонках после запятой 6 знаков и! появился новый столбец индексов с именем Unname 0:
(которую мы должны убрать - займемся этим вопросом ниже)

# Есть два пути как убрать дополнительный столбец индексов - удалисть столбец  методом .drop или  index_col = 0 при чтении файла
# Удаляем вновь появившийся столбец индексов "Unnamed: 0"
# ['Unnamed: 0'] - указывает на столбец ,который нужно удалить
# axis=1 - Помечает столбец в фрейме данных, подлежащие удалению
# inplace=True - Выполняет операцию удаления в том же фрейме данных, 
# а не создает новый объект фрейма данных во время операции удаления.

# df_bp.drop(['Unnamed: 0'], axis=1, inplace=True)

# df_bp.drop() 


# Или можно сделать все тоже самое корече -  Удаляем вновь появившийся столбец индексов "Unnamed: 0" во время чтения excel файла .xlsc c помощбю утилиты:
# index_col = 0


In [4]:
# считываем 1-й файл X_bp.xlsx
df_bp = pd.read_excel(r'C:\Users\grain\Work_folder\Diplom_MGTU\data_diplom\X_bp.xlsx', index_col = 0) 

In [5]:
df_bp.head() # выводим df_bp

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2"
0,1.857143,2030.0,738.736842,30.00,22.267857,100.000000,210.0,70.0,3000.0,220.0
1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0
2,1.857143,2030.0,738.736842,49.90,33.000000,284.615385,210.0,70.0,3000.0,220.0
3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0
4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0


In [6]:
df_bp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1023 entries, 0 to 1022
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Соотношение матрица-наполнитель       1023 non-null   float64
 1   Плотность, кг/м3                      1023 non-null   float64
 2   модуль упругости, ГПа                 1023 non-null   float64
 3   Количество отвердителя, м.%           1023 non-null   float64
 4   Содержание эпоксидных групп,%_2       1023 non-null   float64
 5   Температура вспышки, С_2              1023 non-null   float64
 6   Поверхностная плотность, г/м2         1023 non-null   float64
 7   Модуль упругости при растяжении, ГПа  1023 non-null   float64
 8   Прочность при растяжении, МПа         1023 non-null   float64
 9   Потребление смолы, г/м2               1023 non-null   float64
dtypes: float64(10)
memory usage: 87.9 KB


In [7]:
# теперь запишем наш файл с исходными данными уже в формате .CSV в нашу папку, 
# где находится наш проектный Jupiter_notebook-Diplom_MGTU
# Приводим количество знаков у переменных типа float
# к 3 знакам после запятой float_format="%.3f" для единообразия

df_bp.to_csv('x_bp_new.csv', index=False, float_format="%.3f")

In [8]:
# теперь прочитаем нормальный файл формата .CSV

df_bp_new = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\Diploma1_data\x_bp_new.csv')

In [9]:
# .head() выводим первые 5 строк для проверки.

df_bp_new

# Как видим у разных колонок разное количество знаков после запятой. Мы займемся этим на этапе подготовки данных для машинного обучения

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2"
0,1.857,2030.000,738.737,30.000,22.268,100.000,210.000,70.000,3000.000,220.000
1,1.857,2030.000,738.737,50.000,23.750,284.615,210.000,70.000,3000.000,220.000
2,1.857,2030.000,738.737,49.900,33.000,284.615,210.000,70.000,3000.000,220.000
3,1.857,2030.000,738.737,129.000,21.250,300.000,210.000,70.000,3000.000,220.000
4,2.771,2030.000,753.000,111.860,22.268,284.615,210.000,70.000,3000.000,220.000
...,...,...,...,...,...,...,...,...,...,...
1018,2.271,1952.088,912.856,86.992,20.123,324.775,209.199,73.091,2387.292,125.008
1019,3.444,2050.089,444.733,145.982,19.600,254.215,350.661,72.921,2360.393,117.730
1020,3.281,1972.373,416.837,110.533,23.958,248.423,740.143,74.734,2662.906,236.607
1021,3.705,2066.800,741.476,141.398,19.247,275.780,641.468,74.043,2071.716,197.126


Проделаем ниже такие же манипуляции со вторым файлом из DataSet из папки Diplom_MGTU/data_diplom/X_nup.xlsx

In [10]:
# считываем 2-й файл X_nup.xlsx
# сразу убираем лишний столбец с индексами index_col = 0

df_nup = pd.read_excel(r'C:\Users\grain\Work_folder\Diplom_MGTU\data_diplom\X_nup.xlsx', index_col = 0)
df_nup.tail()

,"Угол нашивки, град",Шаг нашивки,Плотность нашивки
1035,90,8.088111,47.759177
1036,90,7.619138,66.931932
1037,90,9.800926,72.858286
1038,90,10.079859,65.519479
1039,90,9.021043,66.920143


In [11]:
# теперь запишем наш файл с исходными данными уже в формате .CSV в нашу папку, 
# где находится наш проектный Jupiter_notebook-Diplom_MGTU
# и также приводим количес тво знаков у переменных типа float к 3 знакам после запятой float_format="%.3f"

df_nup.to_csv('x_nup_new.csv', index=False, float_format="%.3f")


In [12]:
# теперь прочитаем нормальный файл формата .CSV

df_nup_new = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\Diploma1_data\x_nup_new.csv')
df_nup_new.tail()

# .tail() выводим 5 последних строк для проверки.

,"Угол нашивки, град",Шаг нашивки,Плотность нашивки
1035,90,8.088,47.759
1036,90,7.619,66.932
1037,90,9.801,72.858
1038,90,10.080,65.519
1039,90,9.021,66.920


Таким образом, мы получили 2 файла из нашего исходного DataSet:
df_bp_new.csv (1023 строк на 10 столбцов)
df_nup_new.csv (1040 строк на 3 столбца)
Переходим к следующему Этапу


# Этапа 3 Data Preparetion / Подготовка данных

# Этап 3.1. Select Data / Выборка данных

Два полученных нами файла с данными указывают, что файл df_bp_new.csv (1023 строк на 10 столбцов) является базовым файлом и имет 1023 строки данных по характеристикам различных композитных материалов.
А файл df_nup_new.csv (1040 строк на 3 столбца) имеет на 17 строк больше чем основной файл df_bp_new.csv.

# Этап 3.2. Clean Data/ Очистку данных

Убираем из файла df_nup_new.csv  лишние 17 строк.

name.index - список строк, подлежащих удалению в файле
axis=0 - Помечает строки в фрейме данных, подлежащие удалению
inplace=True - Выполняет операцию удаления в том же фрейме данных, а не создает новый объект фрейма данных во время операции удаления.

df.index[1023:1040] генерирует диапазон строк от 1023 до 1040 (у нас 1039 +1 чтобы вошла и последняя строка). Нижний предел диапазона является включающим, а верхний предел диапазона является исключающим. Это означает, что строки 1023 и 1039 будут удалены.
inplace=True выполняет операцию удаления в том же фрейме данных

In [13]:
# Удаляем из датасета 17 строк, в которых недостаточно данных для обучения.
df_nup_new.drop(df_nup_new.index[1023:1040], axis = 0, inplace=True)

In [14]:
df_nup_new.tail() # проверяем

,"Угол нашивки, град",Шаг нашивки,Плотность нашивки
1018,90,9.076,47.020
1019,90,10.566,53.751
1020,90,4.161,67.630
1021,90,6.313,58.261
1022,90,6.079,77.434


У нас получилась таблица 1023 строки на 3 стобца


# Этап 3.3. Integrate Data / Интеграция файлов

Объеденяем 2 файла в один hw_data_composite с количесвом строк 1023  и добавляем к имеющимся в файле df_bp_new.csv 10 столбцам  еще 3 столбца  из файла df_nup_new.csv.
В итоге должны получимть DataSen размерностью 1023 строк на 13 столбцов

Получим информацию о наших файлах df_bp_new.csv и df_nup_new.csv

In [15]:
# Получаем информацибю о df_bp_new
df_bp_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Соотношение матрица-наполнитель       1023 non-null   float64
 1   Плотность, кг/м3                      1023 non-null   float64
 2   модуль упругости, ГПа                 1023 non-null   float64
 3   Количество отвердителя, м.%           1023 non-null   float64
 4   Содержание эпоксидных групп,%_2       1023 non-null   float64
 5   Температура вспышки, С_2              1023 non-null   float64
 6   Поверхностная плотность, г/м2         1023 non-null   float64
 7   Модуль упругости при растяжении, ГПа  1023 non-null   float64
 8   Прочность при растяжении, МПа         1023 non-null   float64
 9   Потребление смолы, г/м2               1023 non-null   float64
dtypes: float64(10)
memory usage: 80.0 KB


In [16]:
# Получаем информацибю о df_nup_new
df_nup_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Угол нашивки, град  1023 non-null   int64  
 1   Шаг нашивки         1023 non-null   float64
 2   Плотность нашивки   1023 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 24.1 KB


# Этап 3.3. Integrate Data  / Интеграция данных

Объеденяем с помощью pd.merge по способу how='inner' 2 начальных файла в один файл hw_data_composite_main с количесвом строк 1023  и добавляем к имеющимся в файле df_bp_new.csv 10 стоaлбцам  еще 3 столбца  из файла df_nup_new.csv.
В итоге должны получимть DataSet размерностью 1023 строк на 13 столбцов

In [17]:
hw_data_composite_main = pd.merge(df_nup_new, df_bp_new, how='inner', on=None, left_on=None, right_on=None,
                                  left_index=True, right_index=True, sort=True)

In [18]:
# проверяем что получилось
hw_data_composite_main.info() # итоговый DataFrame с размерностью  1023 на 13

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Угол нашивки, град                    1023 non-null   int64  
 1   Шаг нашивки                           1023 non-null   float64
 2   Плотность нашивки                     1023 non-null   float64
 3   Соотношение матрица-наполнитель       1023 non-null   float64
 4   Плотность, кг/м3                      1023 non-null   float64
 5   модуль упругости, ГПа                 1023 non-null   float64
 6   Количество отвердителя, м.%           1023 non-null   float64
 7   Содержание эпоксидных групп,%_2       1023 non-null   float64
 8   Температура вспышки, С_2              1023 non-null   float64
 9   Поверхностная плотность, г/м2         1023 non-null   float64
 10  Модуль упругости при растяжении, ГПа  1023 non-null   float64
 11  Прочность при рас

# !!! Cохраним основной DataFrame в файле hw_data_composit_main.csv на диске в директории Diplom_MGTU чтобы не портить начальные данные. 

In [19]:
hw_data_composite_main.to_csv('hw_data_composite_main.csv', index=False, float_format="%.3f")

In [20]:
# Создадим новый df для дальнейшей работы
data_composite_inspect = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\Diploma1_data\hw_data_composite_main.csv')

In [21]:
data_composite_inspect.head(10)

,"Угол нашивки, град",Шаг нашивки,Плотность нашивки,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2"
0,0,4.0,57.0,1.857,2030.0,738.737,30.00,22.268,100.000,210.0,70.0,3000.0,220.0
1,0,4.0,60.0,1.857,2030.0,738.737,50.00,23.750,284.615,210.0,70.0,3000.0,220.0
2,0,4.0,70.0,1.857,2030.0,738.737,49.90,33.000,284.615,210.0,70.0,3000.0,220.0
3,0,5.0,47.0,1.857,2030.0,738.737,129.00,21.250,300.000,210.0,70.0,3000.0,220.0
4,0,5.0,57.0,2.771,2030.0,753.000,111.86,22.268,284.615,210.0,70.0,3000.0,220.0
5,0,5.0,60.0,2.768,2000.0,748.000,111.86,22.268,284.615,210.0,70.0,3000.0,220.0
6,0,5.0,70.0,2.570,1910.0,807.000,111.86,22.268,284.615,210.0,70.0,3000.0,220.0
7,0,7.0,47.0,2.561,1900.0,535.000,111.86,22.268,284.615,380.0,75.0,1800.0,120.0
8,0,7.0,57.0,3.557,1930.0,889.000,129.00,21.250,300.000,380.0,75.0,1800.0,120.0
9,0,7.0,60.0,3.532,2100.0,1421.000,129.00,21.250,300.000,1010.0,78.0,2000.0,300.0


In [22]:
data_composite_inspect.describe()

,"Угол нашивки, град",Шаг нашивки,Плотность нашивки,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2"
count,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000,1023.000000
mean,44.252199,6.899229,57.153935,2.930358,1975.734881,739.923246,110.570768,22.244387,285.882149,482.731801,73.328566,2466.922854,218.423154
std,45.015793,2.563452,12.350963,0.913224,73.729223,330.231580,28.295897,2.406302,40.943255,281.314699,3.119000,485.627995,59.735918
min,0.000000,0.000000,0.000000,0.389000,1731.765000,2.437000,17.740000,14.255000,100.000000,0.604000,64.054000,1036.857000,33.803000
25%,0.000000,5.080000,49.799500,2.318000,1924.155500,500.047000,92.443500,20.608000,259.066500,266.816500,71.245000,2135.850500,179.627500
50%,0.000000,6.916000,57.342000,2.907000,1977.622000,739.664000,110.565000,22.231000,285.897000,451.864000,73.269000,2459.525000,219.199000
75%,90.000000,8.586000,64.945000,3.552500,2021.374500,961.812500,129.730500,23.962000,313.002000,693.225000,75.357000,2767.193000,257.482000
max,90.000000,14.441000,103.989000,5.592000,2207.773000,1911.536000,198.953000,33.000000,413.273000,1399.542000,82.682000,3848.437000,414.591000


Набор данных содержит многословные названия столбцов с символами, словами в верхнем и нижнем регистре и 
пробелами и опечатками. Чтобы упростить выборку данных по имени столбца,приведем  названия столбцов.
Рекомендуется использовать английский язык,нижний регистр, удалить специальные символы и заменить пробелы символами подчеркивания

In [ ]:
# data_composite_inspect

Перевод  значений str из списка на другой язык (с любого )

Заменим  названия столбцов с русского языка на английский язык

In [23]:
# Функция перевода строковых данных ('слово или предложение') или Списка (list) на другой язык
# передаем параметры:
# list_data - строку в формате 'str' или в виде списка list
# lang_old - необязательный параметр - src - код  исходного языка - по таблие сокращений языков (в нашем случае - 'ru' - русский) 
# lang_new - параметр -dest -код   целевого языка - по таблице сокращений языков (по умолчанию - 'en' - английский)
# Если не указать lang_new - параметр -'dest' - прорамма по умолчанию применит английский язык.
# prt - параметр для печати результатов. Если = True - печать, если Flse - Нет печати

def translate_list_data (data, src, dest, prt = True):
    
    list_old_name = data
    # Создаем пустой список для вывод переведенного списка
    list_new_name = []

    translator = Translator() # вызываем метод Translator()

    for i in list_old_name: # проходим цикл по списку
   
        translated = translator.translate(i, src = 'ru', dest= 'en').text # осуществляем перевод по элемнтам списка
    
        list_new_name.append(translated) # добавляем переведенные элементы  в списое первода

    if prt==True:
        print(f'{list_old_name} -> {list_new_name}')
        
    return list_new_name

In [24]:
list_column =data_composite_inspect.columns.tolist() # Cписок наименований столбцов для передачи в функцию перевода

In [25]:
list_column

['Угол нашивки, град',
 'Шаг нашивки',
 'Плотность нашивки',
 'Соотношение матрица-наполнитель',
 'Плотность, кг/м3',
 'модуль упругости, ГПа',
 'Количество отвердителя, м.%',
 'Содержание эпоксидных групп,%_2',
 'Температура вспышки, С_2',
 'Поверхностная плотность, г/м2',
 'Модуль упругости при растяжении, ГПа',
 'Прочность при растяжении, МПа',
 'Потребление смолы, г/м2']

In [26]:
list_new_name = translate_list_data(list_column,'ru', 'en', prt = True) # Функция перевода названий столбцов ( в данном случае) 

['Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки', 'Соотношение матрица-наполнитель', 'Плотность, кг/м3', 'модуль упругости, ГПа', 'Количество отвердителя, м.%', 'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2', 'Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Потребление смолы, г/м2'] -> ['Patch angle, hail', 'Step of the strip', 'The density of the strip', 'The ratio of the filler matrix', 'Density, kg/m3', 'elasticity module, GPa', 'The number of hardeners, m.%', 'The content of epoxy groups,%_ 2', 'Flash temperature, s_2', 'Surface density, g/m2', 'Elasticity module for stretching, GPa', 'Strapery strength, MPa', 'Resin consumption, g/m2']


In [27]:
print(list_new_name)

['Patch angle, hail', 'Step of the strip', 'The density of the strip', 'The ratio of the filler matrix', 'Density, kg/m3', 'elasticity module, GPa', 'The number of hardeners, m.%', 'The content of epoxy groups,%_ 2', 'Flash temperature, s_2', 'Surface density, g/m2', 'Elasticity module for stretching, GPa', 'Strapery strength, MPa', 'Resin consumption, g/m2']


In [28]:
# Переименуем наименования столбцов в нашем df
# часть взята из https://dev-gang.ru/article/perevod-teksta-s-pomosczu-google-translate-api-v-python-ahgm88wx1k/

def сhangelang_column_name(df, list_new_name, prt = True):
    # функция полчает df  и  заменяет названия столбцов с русского языка на ангийский язык 
    # list_new_name - список новых наименований столбцов
    # list_old_name -  Список старых названий столбцов
  
    list_old_name = list(df.columns)
            
    # Создадим словарь для его подcтановки в метод .rename для замены имен столбцов
    dict1 = {} # создаем пустой словарь

    for i in range(len(list_old_name)):
        dict1[list_old_name[i]] = list_new_name[i]
        df.rename(columns=dict1, inplace=True)

    if prt==True:
        print(dict1)
    return df

In [29]:
сhangelang_column_name(data_composite_inspect, list_new_name)

{'Угол нашивки, град': 'Patch angle, hail', 'Шаг нашивки': 'Step of the strip', 'Плотность нашивки': 'The density of the strip', 'Соотношение матрица-наполнитель': 'The ratio of the filler matrix', 'Плотность, кг/м3': 'Density, kg/m3', 'модуль упругости, ГПа': 'elasticity module, GPa', 'Количество отвердителя, м.%': 'The number of hardeners, m.%', 'Содержание эпоксидных групп,%_2': 'The content of epoxy groups,%_ 2', 'Температура вспышки, С_2': 'Flash temperature, s_2', 'Поверхностная плотность, г/м2': 'Surface density, g/m2', 'Модуль упругости при растяжении, ГПа': 'Elasticity module for stretching, GPa', 'Прочность при растяжении, МПа': 'Strapery strength, MPa', 'Потребление смолы, г/м2': 'Resin consumption, g/m2'}


,"Patch angle, hail",Step of the strip,The density of the strip,The ratio of the filler matrix,"Density, kg/m3","elasticity module, GPa","The number of hardeners, m.%","The content of epoxy groups,%_ 2","Flash temperature, s_2","Surface density, g/m2","Elasticity module for stretching, GPa","Strapery strength, MPa","Resin consumption, g/m2"
0,0,4.000,57.000,1.857,2030.000,738.737,30.000,22.268,100.000,210.000,70.000,3000.000,220.000
1,0,4.000,60.000,1.857,2030.000,738.737,50.000,23.750,284.615,210.000,70.000,3000.000,220.000
2,0,4.000,70.000,1.857,2030.000,738.737,49.900,33.000,284.615,210.000,70.000,3000.000,220.000
3,0,5.000,47.000,1.857,2030.000,738.737,129.000,21.250,300.000,210.000,70.000,3000.000,220.000
4,0,5.000,57.000,2.771,2030.000,753.000,111.860,22.268,284.615,210.000,70.000,3000.000,220.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,90,9.076,47.020,2.271,1952.088,912.856,86.992,20.123,324.775,209.199,73.091,2387.292,125.008
1019,90,10.566,53.751,3.444,2050.089,444.733,145.982,19.600,254.215,350.661,72.921,2360.393,117.730
1020,90,4.161,67.630,3.281,1972.373,416.837,110.533,23.958,248.423,740.143,74.734,2662.906,236.607
1021,90,6.313,58.261,3.705,2066.800,741.476,141.398,19.247,275.780,641.468,74.043,2071.716,197.126


Переведем все названия в нижний регистр для дальнейшего удобного использования названий признаков 

мы можем использовать list comprehension:

In [40]:
data_composite_inspect.columns = [col.lower() for col in data_composite_inspect]
print(data_composite_inspect.columns)

Index(['patch angle, hail', 'step_strip', 'density_strip',
       'ratio_filler_matrix', 'density', 'elasticity_module',
       'number_hardeners', 'content_epoxy_groups', 'flash_temperature',
       'surface_density', 'elasticity_module_stretching', 'strapery_strength',
       'resin_consumption'],
      dtype='object')


Приведем имена столбцов к виду -  уберем пробелы и спецсимволы создав словарь

In [42]:
data_composite_inspect.rename(columns={
        'pattern angle, hail':                  'pattern_angle',
        'step of the strip':                    'step_strip',
        'the density of the strip':             'density_strip',
        'the ratio of the filler matrix':       'ratio_filler_matrix',
        'density, kg/m3':                       'density',
        'elasticity module, gpa':               'elasticity_module',
        'the number of hardeners, m.%':         'number_hardeners',
        'the content of epoxy groups,%_ 2':     'content_epoxy_groups',
        'flash temperature, s_2':               'flash_temperature',
        'surface density, g/m2':                'surface_density',
        'elasticity module for stretching, gpa':'elasticity_module_stretching',
        'strapery strength, mpa':               'strapery_strength',
        'resin consumption, g/m2':              'resin_consumption'
    }, inplace=True)

In [43]:
data_composite_inspect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   patch angle, hail             1023 non-null   int32  
 1   step_strip                    1023 non-null   float32
 2   density_strip                 1023 non-null   float32
 3   ratio_filler_matrix           1023 non-null   float32
 4   density                       1023 non-null   float32
 5   elasticity_module             1023 non-null   float32
 6   number_hardeners              1023 non-null   float32
 7   content_epoxy_groups          1023 non-null   float32
 8   flash_temperature             1023 non-null   float32
 9   surface_density               1023 non-null   float32
 10  elasticity_module_stretching  1023 non-null   float32
 11  strapery_strength             1023 non-null   float32
 12  resin_consumption             1023 non-null   float32
dtypes: 

In [45]:
data_composite_inspect.columns = [
                                 'pattern_angle',
                                 'step_strip',
                                 'density_strip',
                                 'ratio_filler_matrix',
                                 'density',
                                 'elasticity_module',
                                 'number_hardeners',
                                 'content_epoxy_groups',
                                 'flash_temperature',
                                 'surface_density',
                                 'elasticity_module_stretching',
                                 'strapery_strength',
                                 'resin_consumption'
                                  ] #, inplace=True)

In [46]:
list(data_composite_inspect)

['pattern_angle',
 'step_strip',
 'density_strip',
 'ratio_filler_matrix',
 'density',
 'elasticity_module',
 'number_hardeners',
 'content_epoxy_groups',
 'flash_temperature',
 'surface_density',
 'elasticity_module_stretching',
 'strapery_strength',
 'resin_consumption']

# Now we will convert it from 'int' to 'float' type 
# using DataFrame.astype() function
data_composite_inspect['pattern_angle'] = data_composite_inspect['pattern_angle'].astype(float)

In [47]:
data_composite_inspect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pattern_angle                 1023 non-null   int32  
 1   step_strip                    1023 non-null   float32
 2   density_strip                 1023 non-null   float32
 3   ratio_filler_matrix           1023 non-null   float32
 4   density                       1023 non-null   float32
 5   elasticity_module             1023 non-null   float32
 6   number_hardeners              1023 non-null   float32
 7   content_epoxy_groups          1023 non-null   float32
 8   flash_temperature             1023 non-null   float32
 9   surface_density               1023 non-null   float32
 10  elasticity_module_stretching  1023 non-null   float32
 11  strapery_strength             1023 non-null   float32
 12  resin_consumption             1023 non-null   float32
dtypes: 

# Для уменьшения размеров используемой памяти изменим размерность float64 на float32

!!!! Код взят из источника https://github.com/ellavs/python-pandas-optimize-dataframe-memory-usage
Для статьи: https://www.e-du.ru/2021/10/optimize-dataframe-memory-usage.html


# ЗАДАЧА перевести нижние 2 функции в модули и запрашивать их из основной программы

In [48]:
# !!!! Код взят из источника https://github.com/ellavs/python-pandas-optimize-dataframe-memory-usage
# Function optimizes memory usage in dataframe.
# (RU) Функция оптимизации типов в dataframe.

def optimize_memory_usage(df, print_size=True):


# Types for optimization.
    # Типы, которые будем проверять на оптимизацию.
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # Memory usage size before optimize (Mb).
    # (RU) Размер занимаемой памяти до оптимизации (в Мб).
    before_size = df.memory_usage().sum() / 1024**2    
    for column in df.columns:
        column_type = df[column].dtypes
        if column_type in numerics:
            column_min = df[column].min()
            column_max = df[column].max()
            if str(column_type).startswith('int'):
                if column_min > np.iinfo(np.int32).min and column_max < np.iinfo(np.int32).max:
                    df[column] = df[column].astype(np.int32)
                elif column_min > np.iinfo(np.int64).min and column_max < np.iinfo(np.int64).max:
                    df[column] = df[column].astype(np.int64)  
            else:
                if column_min > np.finfo(np.float32).min and column_max < np.finfo(np.float32).max:
                    df[column] = df[column].astype(np.float32)
                else:
                    df[column] = df[column].astype(np.float64)    
    # Memory usage size after optimize (Mb).
    # (RU) Размер занимаемой памяти после оптимизации (в Мб).
    after_size = df.memory_usage().sum() / 1024**2
    if print_size: print('Memory usage size: before {:5.4f} Mb - after {:5.4f} Mb ({:.1f}%).'.format(before_size, after_size, 100 * (before_size - after_size) / before_size))
    return df

def import_data_from_csv(df):
    # Show dataframe info before optimize.
    # (RU) Показать информацию о таблице до оптимизации.
    print('-' * 80)
    print(df.info())
    print('-' * 80)
    # (RU) Оптимизация типов в dataframe.
    df = optimize_memory_usage(df) # используем функцию optimize_memory_usage , созданую выше
    # Show dataframe info after optimize.
    # (RU) Показать информацию о таблице после оптимизации.
    print('-' * 80)
    print(df.info())
    print('-' * 80)
    return df

In [ ]:
# Вызываем функцию import_data_from_csv для уменьшения размерности переменных в файле hw_data_composite_main

data_composite_inspect = import_data_from_csv(data_composite_inspect) # проверяем, размерность изменилась

--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pattern_angle                 1023 non-null   int32  
 1   step_strip                    1023 non-null   float32
 2   density_strip                 1023 non-null   float32
 3   ratio_filler_matrix           1023 non-null   float32
 4   density                       1023 non-null   float32
 5   elasticity_module             1023 non-null   float32
 6   number_hardeners              1023 non-null   float32
 7   content_epoxy_groups          1023 non-null   float32
 8   flash_temperature             1023 non-null   float32
 9   surface_density               1023 non-null   float32
 10  elasticity_module_stretching  1023 non-null   float32
 11  strapery_strength             1023 non-n

Понизив размерность данных в файле с float64 до float32 мы уменьшили размер используемой памяти с 104.0 KB
кВ до 52.1 кВ, т.е на 49,9%

Сохраним data_composite_inspect в директории Diploma1_data для дальнейшей работы

In [50]:
data_composite_inspect.to_csv('data_composite_inspect.csv', index=False, float_format="%.3f")

Дальнейшие процедуры будем проводить в новом ноутбуке Diploma2